In [36]:
import os
import requests
from io import BytesIO
import openai
import pandas as pd
import importlib
import time 


# Process
```mermaid
graph TD
    A(LinkedIn Export) -->|Python| B(Cleaned Posts)
    B -->|OpenAI CLI| C(JSON Format Posts)
    C -->|Curie| D(Summaries)
    B -->|Python| E(Merged Posts + Summaries)
    D -->|Python| E(Merged Posts + Summaries)
    E -->|OpenAI CLI| F(JSON Post + Summaries)
    F -->|Davinci fine tuning| G(Fine Tuned Model)
```

In [7]:
import src.utils.DataCleaning as dc


In [ ]:
importlib.reload(dc)
#testing
shares = dc.loadLinkedInSharesData('./src/local_data/shares.csv')
shares.head()

In [32]:
#build workflow for cleaning
##only need ShareCommentary field
sample_posts = pd.DataFrame(columns=['prompt', 'completion'])
sample_posts['completion'] = shares['ShareCommentary'].str.translate(str.maketrans('', '', '"'))
print(sample_posts['completion'][1])
## output has strange double quotes around each line, eventually we can try to identify intended quotes, but for now, we'll get rid of all of them


If you're dealing with zip codes in your data analysis, here's a quick video that goes over the process of treating numbers like text. 

#licreatoraccelerator #linkedincreator #linkedincreatoraccelerator #datanalysis #datanalytics #yaads #datascience 



In [55]:
# Now a series of 'prompts' asking to summarize the linked in posts as a chatgpt3 prompt.

summarize_prefix = " Summarize the following linkedin post for a chatgpt3 prompt, start prompt with 'A LinkedIn post about': \n "

summary_posts = summarize_prefix + sample_posts['completion'] 


In [56]:
summary_posts[1]

" Summarize the following linkedin post for a chatgpt3 prompt, start prompt with 'A LinkedIn post about': \n If you're dealing with zip codes in your data analysis, here's a quick video that goes over the process of treating numbers like text. \r\n\r\n#licreatoraccelerator #linkedincreator #linkedincreatoraccelerator #datanalysis #datanalytics #yaads #datascience \r\n"

In [38]:
#Create your own local directory and store the key file from openai
key = open("./.config/openai.key").read()

In [73]:
# Load your API key from an environment variable or secret management service
openai.api_key = key

# send a Completion request to count to 100

# def curie_summarize(p):
response = openai.Completion.create(
    model='text-curie-001',
    prompt= summary_posts[1], #p,
    max_tokens=100,
    temperature=0,
    stream=False,  # this time, we set stream=True
)
    # return(response)


# # create variables to collect the stream of events
collected_prompts = []
# # iterate through posts
for post in summary_posts:
    # print(post)
    response = openai.Completion.create(
        model='text-curie-001',
        prompt= f'"{post}"', #p,
        max_tokens=100,
        temperature=0,
        stream=False,  # this time, we set stream=True
    )
#     res = curie_summarize(f'"{post}"')
    collected_prompts.append(response["choices"][0]["text"])  # save the response



In [74]:
sample_posts['prompt'] = collected_prompts


In [78]:
sample_posts.to_json("./src/local_data/shares.jsonl", orient='records', lines=True)

# Variations by platform
Powershell
`$env:OPENAI_API_KEY='{key}' ; penai tools fine_tunes.prepare_data -f ./src/local_data/shares.jsonl -q`

Windows Command Line
`!set OPENAI_API_KEY={key} && openai tools fine_tunes.prepare_data -f ./src/local_data/shares.jsonl -q`

Linux Bash/Shell
`export OPENAI_API_KEY='{key}' ; penai tools fine_tunes.prepare_data -f ./src/local_data/shares.jsonl -q`

In [ ]:
!set OPENAI_API_KEY={key} && openai tools fine_tunes.prepare_data -f ./src/local_data/shares.jsonl -q